In [ ]:
!unzip '/content/drive/My Drive/Copy of shopee-product-detection-dataset' -d dataset

In [ ]:
import tensorflow as tf
import keras 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings; 
import pandas as pd
warnings.simplefilter('ignore')

broken_fnames = []
train_path ='/content/dataset/train/train'
for label in os.listdir(train_path):
    label_path = train_path + '/' + label 
    for filename in os.listdir(label_path):
        if len(filename) > 36:
            broken_fnames.append(label_path + filename)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 128
SEED = 0

def get_set():
    train_path ='/content/dataset/train/train'
    test_path = '/content/dataset/test'

    train_gen = ImageDataGenerator(rescale=1./255, validation_split=0.3)
    train_set = train_gen.flow_from_directory(train_path, target_size=IMAGE_SIZE, \
                                              batch_size=BATCH_SIZE, seed=SEED, \
                                              subset='training')
    val_set = train_gen.flow_from_directory(train_path, target_size=IMAGE_SIZE, \
                                            batch_size=BATCH_SIZE, seed=SEED, \
                                            subset='validation')

    test_gen = ImageDataGenerator(rescale=1./255)
    test_set = test_gen.flow_from_directory(test_path, target_size=IMAGE_SIZE, \
                                             batch_size=BATCH_SIZE, seed=SEED, \
                                             shuffle=False, class_mode=None)
    
    return train_set, val_set, test_set

train_set, val_set, test_set = get_set()

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

def get_model():
    base = MobileNetV2(input_shape=IMAGE_SIZE+(3,), include_top=False, \
                       pooling='avg', weights='none')
    base.trainable = False
    dense = Dense(42, activation='softmax', name='dense')(base.output)

    model = Model(inputs=base.inputs, outputs=dense, name='mobilenetv2')
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    return model

model = get_model()

EPOCHS = 20

hist = model.fit(train_set, epochs=EPOCHS, batch_size=BATCH_SIZE, \
                 validation_data=val_set, shuffle=False)
model.save('/content/drive/My Drive/Shopee/mobilenetv2-e203.hdf5')

def generate_prediction(model, save_name):
    subm = pd.read_csv('/content/dataset/test.csv')
    subm = subm.sort_values(by='filename', ignore_index=True)
    
    fnames = sorted(os.listdir('/content/dataset/test/test/'))
    unbroken_index = np.where(np.vectorize(len)(np.array(fnames)) == 36)[0]
    
    y_pred = model.predict(test_set, batch_size=BATCH_SIZE)
    pred = y_pred.argmax(axis=1)
    pred = pred[unbroken_index]
    subm['category'] = pred
    subm['category'] = subm['category'].apply(lambda x : '%02d' % x) # zero pad
    
    subm.to_csv(save_name, index=False)
    return subm
subm = generate_prediction(model, '/content/drive/My Drive/Shopee/submission-e203.csv')

Found 73796 images belonging to 42 classes.
Found 31596 images belonging to 42 classes.
Found 12192 images belonging to 1 classes.
Epoch 1/20
577/577 [==============================] - 1003s 2s/step - loss: 1.5281 - acc: 0.5901 - val_loss: 1.2836 - val_acc: 0.6509
Epoch 2/20
577/577 [==============================] - 1002s 2s/step - loss: 1.1935 - acc: 0.6739 - val_loss: 1.2358 - val_acc: 0.6641
Epoch 3/20
577/577 [==============================] - 995s 2s/step - loss: 1.1082 - acc: 0.6978 - val_loss: 1.2144 - val_acc: 0.6712
Epoch 4/20
577/577 [==============================] - 1008s 2s/step - loss: 1.0529 - acc: 0.7106 - val_loss: 1.2244 - val_acc: 0.6685
Epoch 5/20
577/577 [==============================] - 1009s 2s/step - loss: 1.0120 - acc: 0.7212 - val_loss: 1.2084 - val_acc: 0.6745
Epoch 6/20
577/577 [==============================] - 1024s 2s/step - loss: 0.9816 - acc: 0.7279 - val_loss: 1.2101 - val_acc: 0.6752
Epoch 7/20
577/577 [==============================] - 1012s 2s/ste

In [ ]:
import tensorflow as tf
import keras 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings; 
import pandas as pd
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 128
SEED = 0
test_path = '/content/dataset/test'
test_gen = ImageDataGenerator(rescale=1./255)
test_set = test_gen.flow_from_directory(test_path, target_size=IMAGE_SIZE, \
                                             batch_size=BATCH_SIZE, seed=SEED, \
                                             shuffle=False, class_mode=None)
def generate_prediction(model, save_name):
    subm = pd.read_csv('/content/dataset/test.csv')
    subm = subm.sort_values(by='filename', ignore_index=True)
    
    fnames = sorted(os.listdir('/content/dataset/test/test/'))
    unbroken_index = np.where(np.vectorize(len)(np.array(fnames)) == 36)[0]
    
    y_pred = model.predict(test_set, batch_size=BATCH_SIZE)
    pred = y_pred.argmax(axis=1)
    pred = pred[unbroken_index]
    subm['category'] = pred
    subm['category'] = subm['category'].apply(lambda x : '%02d' % x) # zero pad
    
    subm.to_csv(save_name, index=False)
    return subm

from tensorflow.keras.models import load_model
model = load_model('/content/drive/My Drive/Shopee/mobilenetv2-e10.hdf5')
subm = generate_prediction(model, '/content/drive/My Drive/Shopee/submission.csv')

subm

Found 12192 images belonging to 1 classes.


,filename,category
0,0004b03ad7eabfb3989727c461310a84.jpg,12
1,0004d34271d0a0253f8a95d0746bae05.jpg,16
2,000d4ad3d0a372fd6eea2fee6bf738ef.jpg,39
3,0012f125ad07123e69468ae1d1d59de4.jpg,22
4,0015831b249b856e569e883a9dc6d4fa.jpg,12
...,...,...
12181,ffef3b0e7505855ba5794ebab9aff719.jpg,09
12182,ffef86f7872cd020992e00b7cf6585c7.jpg,41
12183,fff8fdfc178e3e94414fc10c69271f88.jpg,10
12184,fff9ee1d7f920b920dd603631d24bc07.jpg,23


In [ ]:
import tensorflow as tf
import keras 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings; 
import pandas as pd
subm = pd.read_csv('/content/dataset/test.csv')
subm.to_csv('/content/drive/My Drive/submission.csv', index=False)

In [ ]:
!/opt/bin/nvidia-smi

Fri Jul  3 13:41:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    73W / 149W |   8439MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  